# K-Nearest Neighbors (K-NN)

## Importing the libraries

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
import kfp
from kfp.components import func_to_container_op
import kfp.components as comp

## Importing the dataset

In [0]:
def load_dataset(file_path : comp.InputPath() , train_output_csv: comp.OutputPath()):
    {
        dataset = pd.read_csv('Social_Network_Ads.csv')
        
    }  

In [ ]:
load_and_preprocess_data_op = func_to_container_op(load_dataset)

## Splitting the dataset into the Training set and Test set

In [0]:
from sklearn.model_selection import train_test_split
def train_and_test_split(train_path: comp.InputPath() feat_train_output_csv: comp.OutputPath()):
    {
        X = dataset.iloc[:, :-1].values
        y = dataset.iloc[:, -1].values
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)
    }   

In [ ]:
load_and_preprocess_data_op = func_to_container_op(train_and_test_split)

## Feature Scaling

In [0]:
from sklearn.preprocessing import StandardScaler
def apply_feature_scaling():
    {
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)
    }   

In [ ]:
load_and_preprocess_data_op = func_to_container_op(apply_feature_scaling)

## Training the K-NN model on the Training set

In [11]:
from sklearn.neighbors import KNeighborsClassifier
def apply_model( train_path: comp.InputPath(),feat_test_path : comp.InputPath(), model_path : comp.OutputPath('KNeighborsClassifier')):
    {
        classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
        classifier.fit(X_train, y_train)
        
        
        classifier.save_model(model_path)
    }   

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [ ]:
load_and_preprocess_data_op = func_to_container_op(apply_model)

## Making the Confusion Matrix

In [14]:
from sklearn.metrics import confusion_matrix, accuracy_score
def create_confusion_matrix():
    {
        cm = confusion_matrix(y_test, y_pred)
        print(cm)
        accuracy_score(y_test, y_pred)
    }   

[[64  4]
 [ 3 29]]


0.93

In [ ]:
load_and_preprocess_data_op = func_to_container_op(create_confusion_matrix)

In [ ]:
def vanilla_pipeline(url):
    
    web_downloader_task = web_downloader_op(url=url)

    load_and_preprocess_data_task = load_dataset(file = web_downloader_task.outputs['data'])

    featured_data_task = train_and_test_split(train = load_and_preprocess_data_task.outputs['train_output_csv'])
    
    train_eval_task = apply_feature_scaling(train = load_and_preprocess_data_task.outputs['train_output_csv'] , feat_train = featured_data_task.outputs['feat_train_output_csv'],)
    
    eval_data_task = create_confusion_matrix(test_data = featured_data_task.outputs['feat_train_output_csv'],model = train_eval_task.output)
    

In [ ]:
# Using kfp.Client() to run the pipeline from notebook itself
client = kfp.Client() # change arguments accordingly

# Running the pipeline
client.create_run_from_pipeline_func(
    vanilla_pipeline,
    arguments={
        # Github url to fetch the data. This would change when you clone the repo. Please update the url as per that.
        'url': 'https://github.com/NeoKish/examples/raw/master/house-prices-kaggle-competition/data.zip'
    })